In [6]:
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../..')))

import pandas as pd
import numpy as np
from geopy.distance import geodesic
from scripts.locator import Locator

### accessibility score: 
- distance to airport 
- distance to train stations
- altitude?

In [3]:
df_municipalities = pd.read_csv("../data/processed/filtered_municipalities.csv")
df_airports = pd.read_csv("../data/processed/filtered_airports.csv")
df_railway = pd.read_csv("../data/processed/filtered_trains.csv")

FileNotFoundError: [Errno 2] No such file or directory: '../data/processed/filtered_municipalities.csv'

In [7]:
df_municipalities.head()

NameError: name 'df_municipalities' is not defined

In [ ]:
df_airports.head()

In [ ]:
df_railway.sample(15)

In [ ]:
# Apply the function to each row in the towns DataFrame and create two new columns
# df_municipalities[['closest_distance_airport', 'nearest_airport']] = Locator(df_municipalities, df_airports, 'airport_name').result


In [ ]:
df_municipalities.query("municipality_name_clean == 'amurrio'")

In [ ]:
# df_municipalities[['closest_distance_train', 'nearest_train_station']]  = Locator(df_municipalities, df_railway, 'station_name').result

In [ ]:
df_municipalities

In [ ]:
df_accessibility = df_municipalities[["cmun", "municipality_name", "province", "altitude", "longitude", "latitude", "nearest_airport", "closest_distance", "nearest_train_station", "closest_distance_train"]]

In [9]:
# df_municipalities = pd.read_csv("../data/processed/filtered_municipalities.csv")
df_accessibility = pd.read_csv("../../data/scored/accessibility.csv")
df_hospitals = pd.read_csv("../../data/processed/filtered_hospitals.csv")
df_hospitals

,hospital_name,municipality,cmun,n_beds,type,management,latitude,longitude
0,Red De Salud Mental De Araba (Hospital Psiquiá...,vitoria-gasteiz,10590,207,Psychiatric,Autonomous Community,-2.678612,42.835656
1,Hospital San José,vitoria-gasteiz,10590,63,Medical-surgical,Private Non-Profit,-2.676640,42.849661
2,Hospital Quirónsalud Vitoria,vitoria-gasteiz,10590,26,General,Private Non-Profit,-2.668035,42.849761
3,"Hospital De Cuidados San Onofre, S.L. (Hospita...",vitoria-gasteiz,10590,82,Geriatrics and/or long-term care,Private Non-Profit,-2.680386,42.837582
4,Hospital De Leza,laguardia,10318,63,General,Autonomous Community,-2.638635,42.575989
...,...,...,...,...,...,...,...,...
919,Hospital De Rehabilitacion Psiquiatrica Prisma,zaragoza,50297,50,Psychiatric,Private Non-Profit,-0.841290,41.711449
920,Centro Sanitario Cinco Villas,ejea de los caballeros,50095,32,Other purpose,Autonomous Community,-1.125290,42.126721
921,Hospital Viamed Montecanal,zaragoza,50297,68,General,Private Non-Profit,-0.948738,41.639042
922,Hospital Universitario De Ceuta,ceuta,51001,252,General,Social Security,-5.341853,35.880332


In [ ]:
df_accessibility

In [ ]:
df_accessibility = df_accessibility[["cmun", "municipality_name", "province", "altitude", "longitude", "latitude", "nearest_airport", "closest_distance_airport", "nearest_train_station", "closest_distance_train"]]

In [ ]:
df_accessibility

In [ ]:
df_hospitals_general = df_hospitals.query("type == 'General'")

In [ ]:
# Apply the function to each row in the towns DataFrame and create two new columns
df_accessibility[['closest_distance_hospital', 'nearest_hospital']] = Locator(df_accessibility, df_hospitals_general, 'hospital_name').result

In [ ]:
df_accessibility

In [ ]:
df_municipalities.to_csv("../data/scored/accessibility.csv", index=False)